In [1]:
%load_ext autoreload
%autoreload 2
#import numba
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
plt.style.use('matplotlibrc')

In [2]:
cases = pd.read_csv('cases.csv')['cases'].values
covars = pd.read_csv('covars.csv')
covars = covars[['seas1', 'seas2', 'seas3', 'seas4', 'seas5', 'seas6']].values[:cases.shape[0],:]

 rho           tau         beta1         beta2         beta3         beta4         beta5         beta6            nu         gamma 
 7.617542e-01  2.660769e+02  1.437621e+00  1.290241e+00  1.167740e+00  1.275811e+00  1.361995e+00  1.153047e+00  9.648961e-01  3.500000e+00 
        sigma        theta0         alpha            mu         delta        sig_sq           S_0           E_0           I_0           A_0 
 5.000000e+00  0.000000e+00  2.397300e-03  4.287000e-04  1.433000e-04  1.041514e-01  9.993196e-01  1.834793e-04  4.969569e-04  0.000000e+00 
          R_0         pop_0         betat 
 0.000000e+00  1.091182e+07 -6.717172e-02 
 
 
 beta1 = .02, beta2 = .02, beta3 = .02,
                beta4 = .02, beta5 = .02, beta6 = .02,
                tau = 0.02, rho = 0.02, nu = 0.02, sig_sq = 0.02,
                E_0 = ivp(0.2), I_0 = ivp(0.2)

In [3]:
params = pd.read_csv('params.csv')
params.T

,0
rho,7.617542e-01
tau,2.660769e+02
beta1,1.437621e+00
beta2,1.290241e+00
beta3,1.167740e+00
beta4,1.275811e+00
beta5,1.361995e+00
beta6,1.153047e+00
nu,9.648961e-01
gamma,3.500000e+00


In [109]:
class diffRound(torch.autograd.function.Function):
    @staticmethod
    def forward(ctx, input):
        ctx.input = input
        return torch.round(input).int()

    @staticmethod
    def backward(ctx, grad_output):
        grad_input = grad_output.clone()
        return grad_input
    
class Haiti1(nn.Module):
    def __init__(self, covars, cases, params, dt):
        super().__init__()
        self.cases = torch.tensor(cases)
        self.covars = torch.tensor(covars)
        
        #trainable params
        self.betas = nn.Parameter(torch.tensor([params['beta1'], params['beta2'], 
                                                params['beta3'], params['beta4'], 
                                                params['beta5'], params['beta6']]).squeeze())
        self.betat = nn.Parameter(torch.tensor(params['betat']))
        self.rho = nn.Parameter(torch.tensor(params['rho']))
        self.tau = nn.Parameter(torch.tensor(params['tau']))
        self.nu = nn.Parameter(torch.tensor(params['nu']))
        self.sig_sq = nn.Parameter(torch.tensor(params['sig_sq']))
        self.E_0 = nn.Parameter(torch.tensor(params['E_0']))
        self.I_0 = nn.Parameter(torch.tensor(params['I_0']))
        
        self.S = torch.tensor(params['S_0']).double() * torch.tensor(params['pop_0'])
        self.E = self.E_0 * torch.tensor(params['pop_0'])
        self.I = self.I_0 * torch.tensor(params['pop_0'])
        self.R = torch.tensor(params['R_0']).double() * torch.tensor(params['pop_0'])
        self.incid = 0
        self.foival = 0
        
        self.mu = torch.tensor(params['mu'])
        self.delta = torch.tensor(params['delta'])
        self.sigma = torch.tensor(params['sigma'])
        self.gamma = torch.tensor(params['gamma'])
        self.alpha = torch.tensor(params['alpha'])
        self.t = 0
        self.dt = dt
        
    #@torch.jit.script_method
    def reulermultinom(self, N: int, rates, dt: float, save=False):
        sumrates = torch.sum(rates)
        p0 = torch.exp(-sumrates * dt)
        m = torch.distributions.Multinomial(total_count=N, probs=torch.cat([p0.reshape(1), (1-p0)*rates/sumrates]))
        samp = m.sample()
        if save:
            self.s_m = m
            self.s_num = samp
        return samp
        
    
    #@torch.jit.script_method
    def step(self):
        pop = self.S + self.E + self.I + self.R
        births = torch.distributions.Poisson(self.mu * self.dt * pop).sample() #* pop
        beta = torch.exp(torch.dot(self.betas, self.covars[self.t]) + (self.betat-215)/(430-215))
        self.sig_mod = torch.distributions.Gamma(self.sig_sq, self.dt)
        self.sig_num = self.sig_mod.sample()
        foi = torch.pow(self.I, self.nu) * beta / pop * self.sig_num / self.dt
        
        s_rates = torch.cat([foi, self.delta])
        e_rates = torch.cat([self.sigma, self.delta])
        i_rates = torch.cat([self.gamma, self.delta])
        r_rates = torch.cat([self.alpha, self.delta])
        
        s_num = self.reulermultinom(int(self.S), s_rates, self.dt, True) if int(self.S) > 0 else torch.tensor([0,0,0]).double()
        e_num = self.reulermultinom(int(self.E), e_rates, self.dt) if int(self.E) > 0 else torch.tensor([0,0,0]).double()
        i_num = self.reulermultinom(int(self.I), i_rates, self.dt) if int(self.I) > 0 else torch.tensor([0,0,0]).double()
        r_num = self.reulermultinom(int(self.R), r_rates, self.dt) if int(self.R) > 0 else torch.tensor([0,0,0]).double()
        
        self.S = self.S.clone() + torch.tensor([0, -1, -1]).double().T @ s_num + r_num[1] + births
        self.E = self.E.clone() + torch.tensor([0, -1, -1]).double().T @ e_num + s_num[1]
        self.I = self.I.clone() + torch.tensor([0, -1, -1]).double().T @ i_num + e_num[1]
        self.R = self.R.clone() + torch.tensor([0, -1, -1]).double().T @ r_num + i_num[1]
        
        self.incid += e_num[0]
        self.foival += foi
        
    def rprocess(self):
        for i in range(int(1/self.dt)):
            self.step()
        self.t += 1
        
    def rmeasure(self):
        m = torch.distributions.negative_binomial.NegativeBinomial(
            total_count=self.tau, probs = self.tau/(self.tau+(self.rho * self.incid)))
        return m.sample()
    
    def dmeasure(self, case=None):
        if case is None:
            case = self.rmeasure()
        m = torch.distributions.negative_binomial.NegativeBinomial(
            total_count=self.tau, probs = self.tau/(self.tau+(self.rho * self.incid)))
        return m.log_prob(case)
                

In [224]:
import copy
from tqdm import tqdm
import torch.multiprocessing as mp

def innerpfilter(particle):
    particle.rprocess()
    return particle

def copyparams(source, target):
    target.load_state_dict(source.state_dict())
    target.S = source.S.clone()
    target.E = source.E.clone()
    target.I = source.I.clone()
    target.R = source.R.clone()
    target.incid = source.incid.clone()
    target.foival = source.foival.clone()
    
    
def pfilter(constructor = Haiti1, covars = covars, cases = cases, params = params, dt = 1/7, Np = 100):
    loss1 = 0
    loss2 = 0
    loss3 = 0
    loglik = None
    particles = [[Haiti1(covars, cases, params, dt) for j in range(Np)]]
        
    for idx, case in tqdm(enumerate(cases)):
        case = torch.tensor(case)
        
        for particle in particles[idx]:
            particle.rprocess()
        print([particle.S for particle in particles[idx]])
        print([id(particle) for particle in particles[idx]])
            
        weights = [particle.dmeasure(case) for particle in particles[idx]]
        weights = torch.cat(weights)

        m = torch.distributions.Multinomial(total_count = Np, logits = weights)
        counts = m.sample().int()
        reward = torch.log(torch.mean(torch.exp(weights)))

        loss1 += -m.log_prob(counts) * reward # reward here
        for mod1 in particles[idx]:
            loss2 += -mod1.s_m.log_prob(mod1.s_num) * reward / Np
            loss3 += -mod1.sig_mod.log_prob(mod1.sig_num) * reward / Np
        loglik = reward if loglik is None else loglik + reward
        
        newparticles = [Haiti1(covars, cases, params, dt) for j in range(Np)]
        for i, count in enumerate(counts):
            for j in range(count):
                copyparams(particles[idx][i], newparticles[j])
        particles.append(newparticles)
        
    lossfilter = loss1 + loss2 + loss3
    return lossfilter, loglik, particles

mod1 = Haiti1(covars, cases, params, 1/7)
lossfilter, loglik, particles = pfilter(Haiti1, covars[:5], cases[:5], params, 1/7, Np = 3)            

1it [00:02,  2.65s/it]

[tensor([10904842.2025], dtype=torch.float64), tensor([10869768.2025], dtype=torch.float64), tensor([10903181.2025], dtype=torch.float64)]
[5091411808, 5090074480, 5090074432]


2it [00:05,  2.66s/it]

[tensor([10884585.2025], dtype=torch.float64), tensor([10906576.2025], dtype=torch.float64), tensor([10898639.2025], dtype=torch.float64)]
[6220310080, 6220311472, 6220312240]


3it [00:07,  2.67s/it]

[tensor([10899244.2025], dtype=torch.float64), tensor([10909401.2025], dtype=torch.float64), tensor([10909675.2025], dtype=torch.float64)]
[6220311856, 6220309360, 5090216256]


4it [00:10,  2.67s/it]

[tensor([10912677.2025], dtype=torch.float64), tensor([10907862.2025], dtype=torch.float64), tensor([10911991.2025], dtype=torch.float64)]
[6220311760, 5090214864, 5090217840]


5it [00:13,  2.66s/it]

[tensor([10915826.2025], dtype=torch.float64), tensor([10915714.2025], dtype=torch.float64), tensor([10915838.2025], dtype=torch.float64)]
[5090217216, 5090214096, 5090214384]


In [214]:
print([p for p in mod1.parameters()])

[Parameter containing:
tensor([1.4376, 1.2902, 1.1677, 1.2758, 1.3620, 1.1530], dtype=torch.float64,
       requires_grad=True), Parameter containing:
tensor([-0.0672], dtype=torch.float64, requires_grad=True), Parameter containing:
tensor([0.7618], dtype=torch.float64, requires_grad=True), Parameter containing:
tensor([266.0769], dtype=torch.float64, requires_grad=True), Parameter containing:
tensor([0.9649], dtype=torch.float64, requires_grad=True), Parameter containing:
tensor([0.1042], dtype=torch.float64, requires_grad=True), Parameter containing:
tensor([0.0002], dtype=torch.float64, requires_grad=True), Parameter containing:
tensor([0.0005], dtype=torch.float64, requires_grad=True)]


In [208]:
particles[0][0].tau.grad

tensor([0.0914], dtype=torch.float64)

In [222]:
torch.autograd.set_detect_anomaly(True)
optim = torch.optim.Adam(particles[0][-1].parameters(), lr=5000)
lossfilter.backward()
#loglik.backward()

In [223]:
optim.step()
print([p for p in particles[0][-1].parameters()])

[Parameter containing:
tensor([5001.4374, 5001.2901, 5001.1668, 5001.2234, 5001.3096, 5001.1521],
       dtype=torch.float64, requires_grad=True), Parameter containing:
tensor([4999.9172], dtype=torch.float64, requires_grad=True), Parameter containing:
tensor([2666.8420], dtype=torch.float64, requires_grad=True), Parameter containing:
tensor([228.5057], dtype=torch.float64, requires_grad=True), Parameter containing:
tensor([5000.9649], dtype=torch.float64, requires_grad=True), Parameter containing:
tensor([5000.1041], dtype=torch.float64, requires_grad=True), Parameter containing:
tensor([-4999.9997], dtype=torch.float64, requires_grad=True), Parameter containing:
tensor([5000.0005], dtype=torch.float64, requires_grad=True)]


## Stochastic Gradient Testing

This checks if the gradients flow through all parameters, using the log-likelihood given by `dmeasure` after a single step. If gradients indeed do propagate, you should expect to see very very high values.

In [220]:
mod1 = Haiti1(covars, cases, params, 1/7)
mod1.rprocess()
optim = torch.optim.Adam(mod1.parameters(), lr=5000) #turns out, only differentiable wrt rho and tau. Oh well.
loss = -mod1.dmeasure()
loss.backward(retain_graph=True)
loss2 = mod1.s_m.log_prob(mod1.s_num) * loss
loss2.backward(retain_graph=True)
loss3 = mod1.sig_mod.log_prob(mod1.sig_num) * loss
loss3.backward()
optim.step()
print([p for p in mod1.parameters()])

[Parameter containing:
tensor([-4998.5624, -4998.7098, -4998.8322, -4998.7230, -4998.6368, -4998.8469],
       dtype=torch.float64, requires_grad=True), Parameter containing:
tensor([-5000.0668], dtype=torch.float64, requires_grad=True), Parameter containing:
tensor([-4999.2382], dtype=torch.float64, requires_grad=True), Parameter containing:
tensor([5266.0764], dtype=torch.float64, requires_grad=True), Parameter containing:
tensor([-4999.0351], dtype=torch.float64, requires_grad=True), Parameter containing:
tensor([-4999.8958], dtype=torch.float64, requires_grad=True), Parameter containing:
tensor([5000.0002], dtype=torch.float64, requires_grad=True), Parameter containing:
tensor([-4999.9995], dtype=torch.float64, requires_grad=True)]


[Parameter containing:
tensor([-4998.5622, -4998.7096, -4998.8313, -4998.6720, -4998.5859, -4998.8460],
       dtype=torch.float64, requires_grad=True), Parameter containing:
tensor([-5000.0516], dtype=torch.float64, requires_grad=True), Parameter containing:
tensor([5000.7618], dtype=torch.float64, requires_grad=True), Parameter containing:
tensor([-4733.9230], dtype=torch.float64, requires_grad=True), Parameter containing:
tensor([-4999.0351], dtype=torch.float64, requires_grad=True), Parameter containing:
tensor([-4999.8958], dtype=torch.float64, requires_grad=True), Parameter containing:
tensor([5000.0001], dtype=torch.float64, requires_grad=True), Parameter containing:
tensor([-4999.9995], dtype=torch.float64, requires_grad=True)]


In [308]:
print([ p for p in mod1.parameters()])

[Parameter containing:
tensor([1.4376, 1.2902, 1.1677, 1.2758, 1.3620, 1.1530], dtype=torch.float64,
       requires_grad=True), Parameter containing:
tensor([-0.0672], dtype=torch.float64, requires_grad=True), Parameter containing:
tensor([0.7618], dtype=torch.float64, requires_grad=True), Parameter containing:
tensor([266.0769], dtype=torch.float64, requires_grad=True), Parameter containing:
tensor([0.9649], dtype=torch.float64, requires_grad=True), Parameter containing:
tensor([0.1042], dtype=torch.float64, requires_grad=True), Parameter containing:
tensor([0.0002], dtype=torch.float64, requires_grad=True), Parameter containing:
tensor([0.0005], dtype=torch.float64, requires_grad=True)]


In [ ]:
     self.betas = nn.Parameter(torch.tensor([params['beta1'], params['beta2'], 
                                                params['beta3'], params['beta4'], 
                                                params['beta5'], params['beta6']]).squeeze())
        self.betat = nn.Parameter(torch.tensor(params['betat']))
        self.rho = nn.Parameter(torch.tensor(params['rho']))
        self.tau = nn.Parameter(torch.tensor(params['tau']))
        self.nu = nn.Parameter(torch.tensor(params['nu']))
        self.sig_sq = nn.Parameter(torch.tensor(params['sig_sq']))
        self.E_0 = nn.Parameter(torch.tensor(params['E_0']))
        self.I_0 = nn.Parameter(torch.tensor(params['I_0']))

In [270]:
mod1.nu.gradient()

AttributeError: 'Parameter' object has no attribute 'gradient'

In [157]:
mod1.rprocess()

In [158]:
mod1.R

tensor([152320.], dtype=torch.float64)

In [89]:
mod1.S

tensor([10904841.2025], dtype=torch.float64)

In [ ]:
  incids <- c("incid += Etrans[0]; \n ")
  foi_val <- "foival += foi; \n "
  str0 <- "Str0 += Strans[0]; \n "
  sin <- c("Sin += Rtrans[0] + births; \n ")
  sout <- c("Sout += Strans[0] + Strans[1]; \n ")
  last <- c(foi_val, str0, sin)


 ## dmeasure
  dmeas <- Csnippet("
    if (ISNA(cases)) {
      lik = (give_log) ? 0 : 1;
    } else {
      lik = dnbinom_mu(cases, tau, rho*incid, give_log);
    }
  ")

  ## rmeasure
  rmeas <- Csnippet(
    "cases = rnbinom_mu(tau, rho*incid);
    if (cases > 0.0) {
      cases = nearbyint(cases);
    } else {
      cases = 0.0;
    }